# Risk Analysis and Profit Optimization for Oil Well Development in Multiple Regions

Our client is the company "RosGosOil".  We need to determine where to drill a new oil well.

You have been provided with oil samples from three regions: in each region, there are 10,000 oil fields where the quality of the oil and the volume of its reserves have been measured. Build a machine learning model that will help determine the region where extraction will bring the highest profit. Analyze the potential profit and risks using the Bootstrap technique.

Steps for selecting a location:

- In the selected region, search for oil fields and determine the feature values for each field.
- Build a model and estimate the volume of reserves.
- Select the fields with the highest estimated values. The number of fields depends on the company's budget and the - cost of developing one well.
- Profit is equal to the total profit from the selected oil fields.

## Data preprocessing

Import of necessary libraries

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st 

from sklearn.model_selection import train_test_split

import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression 

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_classification
pd.options.mode.chained_assignment = None

state = np.random.RandomState(2020)



Let's save the datases in separate pandas dataframes: 

In [ ]:
data0 = pd.read_csv('geo_data_0.csv')  
data1 = pd.read_csv('geo_data_1.csv')
data2 = pd.read_csv('geo_data_2.csv')

In [ ]:
print(data0.info(),data1.info(),data2.info())

Check for the duplicates:

In [ ]:
data0[data0.duplicated(subset = ['id'])]

In [ ]:
data1[data1.duplicated(subset = ['id'])]

In [ ]:
data2[data2.duplicated(subset = ['id'])]

There are very few duplicates, so they will not affect the model. Let's remove the 'id' column from each dataset: this feature will not help train the model.

In [ ]:
data0=data0.drop(['id'], axis=1)
data0.head()

In [ ]:
data1=data1.drop(['id'], axis=1)
data1.head()

In [ ]:
data2=data2.drop(['id'], axis=1)
data2.head()

We will split the data into training and validation sets and extract the target variable. Since we have three datasets, for convenience, we will create separate functions for each action.

In [ ]:
def split_data(data):  
    data_train, data_valid = train_test_split(data, test_size=0.25, random_state=12345, shuffle = True)
    return data_train, data_valid

# and a function for 'features' and 'target'
def get_features(data, field):
    return data.drop([field], axis = 1)
def get_target(data, field):
    return data[field]

def ftr_trgt_split(data, target_field):
    features_train = get_features(data, target_field)
    target_train = get_target(data, target_field)
    return features_train, target_train

And now apply the functions to the dataframe

In [ ]:
data0_train, data0_valid = split_data(data0)

features0_train, target0_train = ftr_trgt_split(data0_train, 'product')
features0_valid, target0_valid = ftr_trgt_split(data0_valid, 'product')

In [ ]:
data1_train, data1_valid = split_data(data1)

features1_train, target1_train = ftr_trgt_split(data1_train, 'product')
features1_valid, target1_valid = ftr_trgt_split(data1_valid, 'product')

In [ ]:
data2_train, data2_valid = split_data(data2)

features2_train, target2_train = ftr_trgt_split(data2_train, 'product')
features2_valid, target2_valid = ftr_trgt_split(data2_valid, 'product')

Now it's time to scale the data, will create a separate function

In [ ]:
def df_scaler(data_train, data_valid):
    numeric = data_train.columns

    scaler = StandardScaler()
    scaler.fit(data_train[numeric])
    data_train[numeric] = scaler.transform(data_train[numeric])
    data_valid[numeric] = scaler.transform(data_valid[numeric])
    return data_train, data_valid

In [ ]:
features0_train, features0_valid = df_scaler(features0_train, features0_valid)
features1_train, features1_valid = df_scaler(features1_train, features1_valid)
features2_train, features2_valid = df_scaler(features2_train, features2_valid)

So, we have loaded and examined the data, ensuring that there are no missing values. Some duplicates were found in the `id` column. We decided to remove this feature as the unique objects would not help the model learn.

Next, we divided the data for each region into training and validation sets in a 75:25 ratio and extracted the target variable.

Finally, we performed feature scaling.

## Model Training and Evaluation
According to the project's specifications, we are only able to work with linear regression for the given data.

Let's create functions for model training, prediction, and obtaining statistical metrics. We will organize the results in a final table.

In [ ]:
def model_predict(features_train, target_train, features_valid, target_valid):
    scaler = StandardScaler()

    scaler.fit(features_train)

    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)

    model = LinearRegression()
    model.fit(features_train, target_train)

    predicted_valid = model.predict(features_valid)
    return model.score(features_valid, target_valid), predicted_valid

Region 0

In [ ]:
model_0_score, predicted_valid_0 = model_predict(features_train=features0_train,
                                                 target_train=target0_train,
                                                 features_valid=features0_valid,
                                                 target_valid=target0_valid)

Region 1

In [ ]:
model_1_score, predicted_valid_1 = model_predict(features_train=features1_train,
                                                 target_train=target1_train,
                                                 features_valid=features1_valid,
                                                 target_valid=target1_valid)

region 2

In [ ]:
model_2_score, predicted_valid_2 = model_predict(features_train=features2_train,
                                                 target_train=target2_train,
                                                 features_valid=features2_valid,
                                                 target_valid=target2_valid)

Separate tab;e for the results

In [ ]:
target0_predict = pd.concat([target0_valid.reset_index(drop=True), pd.DataFrame(predicted_valid_0)], axis=1)
target1_predict = pd.concat([target1_valid.reset_index(drop=True), pd.DataFrame(predicted_valid_1)], axis=1)
target2_predict = pd.concat([target2_valid.reset_index(drop=True), pd.DataFrame(predicted_valid_2)], axis=1)

target0_predict = target0_predict.rename(columns={0:'predicted_product'})
target1_predict = target1_predict.rename(columns={0:'predicted_product'})
target2_predict = target2_predict.rename(columns={0:'predicted_product'})

In [ ]:
target1_valid.sum()


In [ ]:
model_results = pd.DataFrame({'Real Reserves of Resources'        :[target0_valid.sum(),
                                                               target1_valid.sum(),
                                                               target2_valid.sum()],
                              'Predicted Reserves of Resources'      : [predicted_valid_0.sum(),
                                                                predicted_valid_1.sum(),
                                                                predicted_valid_2.sum()],
                              'Avg Predicted Reserves of Resources' : [predicted_valid_0.mean(),
                                                                predicted_valid_1.mean(),
                                                                predicted_valid_2.mean()],
                              'RMSE'                         : [(mean_squared_error(target0_valid, predicted_valid_0))**0.5,
                                                                (mean_squared_error(target1_valid, predicted_valid_1))**0.5,
                                                                (mean_squared_error(target2_valid, predicted_valid_2))**0.5],
                              'R2-score'                     : [model_0_score,
                                                                model_1_score,
                                                                model_2_score]
}
)

model_results


We prepared the linear regression model, trained it, and obtained the main statistical metrics. Let's proceed with the calculation of profit and risks according to the given conditions. The minimum value of RMSE was obtained in Region 1.

## Profit Calculation Preparation
During the exploration of a region, 500 points are investigated, from which the top 200 are selected for development using machine learning.

The budget for well development in the province is 10 billion rubles.
At current prices, one barrel of resources generates a revenue of 450 rubles.
The revenue per unit of product is 450,000 rubles, as the volume is given in thousands of barrels.

Let's create new variables:

In [ ]:
BUDGET = 10000000000
PRICE = 450000
OIL_SELECTED = 200
VALUE_TRESHOLD = BUDGET/PRICE/OIL_SELECTED 
PROVINCE_TRESHOLD=BUDGET/PRICE
print('Break-even volume of resources for developing a new well: ', VALUE_TRESHOLD)
print('Break-even volume of resources for developing a new province: ', PROVINCE_TRESHOLD)


Thus, for a break-even development of a well, the volume of resources should be at least 112 units, while for the region it should be at least 2,223 units.

## Profit and risk calculation

Let's create a function for calculating profits:

In [ ]:
def material_sum_revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    material_sum = selected.sum()
    material_revenue = material_sum*PRICE - BUDGET
    return pd.DataFrame({'material_sum':[material_sum], 'material_revenue':[material_revenue]})

Create a bootstrap calculation function: 

In [ ]:
def bootstrap(target_valid, probabilities_valid, hole_count):
    region_boot = pd.DataFrame()
    for i in range(1000):
        target_subsample = target_valid.sample(n=hole_count, replace=True, random_state=state)
        probs_subsample = probabilities_valid[target_subsample.index]
        region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
    return  region_boot

Let's proceed with the distribution of resource reserves and profits for the three provinces using the bootstrap function.

In [ ]:
region_0_boot = bootstrap(target_valid=target0_predict['product'],
                          probabilities_valid=target0_predict['predicted_product'],
                          hole_count=500)

region_1_boot = bootstrap(target_valid=target1_predict['product'],
                            probabilities_valid=target1_predict['predicted_product'],
                            hole_count=500)

region_2_boot = bootstrap(target_valid=target2_predict['product'],
                            probabilities_valid=target2_predict['predicted_product'],
                            hole_count=500)

In [ ]:
print(region_0_boot,region_1_boot,region_2_boot)

Let's calculate the confidence intervals by declaring a function:

In [ ]:
def get_confidence_interval(region_boot):
    return (region_boot.material_revenue.quantile(0.025), region_boot.material_revenue.quantile(0.975))

In [ ]:
confidence_interval_region_0 = get_confidence_interval(region_0_boot)
confidence_interval_region_1 = get_confidence_interval(region_1_boot)
confidence_interval_region_2 = get_confidence_interval(region_2_boot)

Provicial risk evaluation

Let's declare a function called `risk_coeff` that returns the risk coefficient in percentage:

In [ ]:
def risk_coeff(region_boot):
    low_treshold_count = len(region_boot.query('material_revenue <= 0'))
    return low_treshold_count/len(region_boot) * 100

In [ ]:
risk_region_0 = risk_coeff(region_0_boot)
risk_region_1 = risk_coeff(region_1_boot)
risk_region_2 = risk_coeff(region_2_boot)

Let's create a final table that assesses the risk of investments made in the development of each province:

In [ ]:
region_final = pd.DataFrame({'Avg provicial profit, RUB.'      : [region_0_boot.material_revenue.mean(),
                                                                     region_1_boot.material_revenue.mean(),
                                                                     region_2_boot.material_revenue.mean()],
                             '95% conf. interval, RUB.' : [confidence_interval_region_0,
                                                                     confidence_interval_region_1,
                                                                     confidence_interval_region_2],
                             'Probability of Losses, %'             : [risk_region_0,
                                                                     risk_region_1,
                                                                     risk_region_2]}, index=['Province 1','Province 2','Province 3'])
region_final

## Conclusion

Based on the results of the project, we can recommend proceeding with the development of the third region(province). Despite having the maximum explored reserves, the model demonstrated a minimal risk coefficient. However, it should be noted that the average profit in the second region was higher, albeit with a higher risk coefficient. It would be prudent to request additional expert assessment of the provinces by geologists.